In [ ]:
import pandas as pd

# Load the name.basics.tsv file
name_basics = pd.read_csv('name.basics.tsv.gz', sep='\t', compression='gzip', na_values='\\N')

# Load the title.crew.tsv file
title_crew = pd.read_csv('title.crew.tsv.gz', sep='\t', compression='gzip', na_values='\\N')

In [ ]:
import pandas as pd

# Initialize an empty DataFrame for storing results
crew_table_grouped = pd.DataFrame()

# Process the file in chunks
chunk_size = 100000  # Adjust chunk size based on memory constraints
for chunk in pd.read_csv('title.crew.tsv.gz', sep='\t', compression='gzip', na_values='\\N', chunksize=chunk_size):
    # Expand directors and writers for the current chunk
    chunk['directors'] = chunk['directors'].fillna('').str.split(',')
    chunk['writers'] = chunk['writers'].fillna('').str.split(',')

    # Explode directors and writers
    directors_exploded = chunk.explode('directors').merge(
        name_basics[['nconst', 'primaryName']], left_on='directors', right_on='nconst', how='left'
    ).rename(columns={'primaryName': 'director_name'})

    writers_exploded = chunk.explode('writers').merge(
        name_basics[['nconst', 'primaryName']], left_on='writers', right_on='nconst', how='left'
    ).rename(columns={'primaryName': 'writer_name'})

    # Group by tconst
    directors_grouped = directors_exploded.groupby('tconst').agg({
        'director_name': lambda x: ', '.join(x.dropna())
    }).reset_index()

    writers_grouped = writers_exploded.groupby('tconst').agg({
        'writer_name': lambda x: ', '.join(x.dropna())
    }).reset_index()

    # Merge results for the current chunk
    chunk_grouped = directors_grouped.merge(writers_grouped, on='tconst', how='outer').fillna('')

    # Append to the final DataFrame
    crew_table_grouped = pd.concat([crew_table_grouped, chunk_grouped], ignore_index=True)

# Save the processed data to a CSV
crew_table_grouped.to_csv('crew_table_with_directors_and_writers.csv', index=False)

In [ ]:
crew_table_grouped

,tconst,director_name,writer_name
0,tt0000001,William K.L. Dickson,
1,tt0000002,Émile Reynaud,
2,tt0000003,Émile Reynaud,
3,tt0000004,Émile Reynaud,
4,tt0000005,William K.L. Dickson,
...,...,...,...
10618816,tt9916848,Hamdi Alkan,"Yelda Acikgoz, Hamdi Alkan, Birol Elginöz, Nur..."
10618817,tt9916850,Hamdi Alkan,"Yelda Acikgoz, Hamdi Alkan, Birol Elginöz, Bir..."
10618818,tt9916852,Hamdi Alkan,"Yelda Acikgoz, Hamdi Alkan, Birol Elginöz, Nur..."
10618819,tt9916856,Johan Planefeldt,Johan Planefeldt


In [ ]:
# Save the filtered and grouped data to a smaller file
crew_table_grouped.to_csv('crew_table.csv', index=False)

In [ ]:
import pandas as pd

# Load the name.basics.tsv file
name_basics = pd.read_csv('name.basics.tsv.gz', sep='\t', compression='gzip', na_values='\\N')

In [ ]:
# Filter rows where primaryProfession contains 'actor' or 'actress'
filtered_df = name_basics[name_basics['primaryProfession'].str.contains('actor|actress', na=False)]

# Expand 'knownForTitles' column into multiple rows
filtered_df = filtered_df.assign(knownForTitles=filtered_df['knownForTitles'].str.split(',')).explode('knownForTitles')

# Drop rows where primaryName is NaN
filtered_df = filtered_df.dropna(subset=['primaryName'])

# Group by knownForTitles and collect names of actors/actresses
result = filtered_df.groupby('knownForTitles')['primaryName'].apply(lambda names: ', '.join(names.unique())).reset_index()

# Rename columns for clarity
result.columns = ['movieid', 'actors']

# Display the first few rows of the result
print(result.head())

     movieid                                             actors
0  tt0000007                                     Peter Courtney
1  tt0000009  Blanche Bayliss, William Courtenay, Chauncey D...
2  tt0000014                       François Clerc, Benoît Duval
3  tt0000018                                           Delaware
4  tt0000026                                     Antoine Féraud


In [ ]:
# Remove 'tt' prefix from movieid column
# result['movieid'] = result['movieid'].str.replace('tt', '', regex=False)
result['movie_id'] = result['movieid'].str.replace('tt', '')
result.drop(columns=['movieid'], inplace=True)

# Reorder the columns
result = result[['movie_id', 'actors']]

# Display the first few rows of the result
print(result.head())

  movie_id                                             actors
0  0000007                                     Peter Courtney
1  0000009  Blanche Bayliss, William Courtenay, Chauncey D...
2  0000014                       François Clerc, Benoît Duval
3  0000018                                           Delaware
4  0000026                                     Antoine Féraud


In [ ]:
# Save the result to a new CSV file
result.to_csv('movies_with_actors_actresses.csv', index=False)